# Migrating Local Mongo DB to AWS

This is actually probably a part 4 in a longer series around how I went from inception to deployment of my in game win probability app for NBA games, the problem is I spent an entire weekend learning about how to deploy things to AWS and thought I had to write about it to others in a similar predicament and before I forgot

### MongoDB to AWS


AWS has two options for noSQL db's, dynamoDB and DocumentDB
Currently I am exploring how to set up dynamoDB but on my way to this point I figured out how to set up DocumentDB and it was kind of long to be honest

### Difference between DocumentDB and DynamoDB

DynamoDB is Serverless where as DocumentDB is not.
In terms of dollar value (which is what we all care about at the end of the day). DyanmoDB is pay per use/as you use resources and DocumentDB is pay hourly.

Also querying the databases are different - docDB can be intereacted with the mongodb driver where as dynamoDB has it's own api.

## Setting up Document DB

#### Steps

- 1) okay look yeah you're going to need an AWS account, this shit ain't free (well the free stuff is free but documentDB doesn't have a free tier) - (writing this after my first day of docDB charges and damn re-reading this hurt my soul) 
- 2) Access, AWS handles access through the IAM - Identity and Access Manager. You will need to create a new user and add the appropriate policies to it. AmazonDocDBFullAccess.
- 3) Create a VPC for your documentDB 
- 4) Create an EC2 instance 
    - why? okay so heres the annoying part about using documentDB it can only interact with other aws services within the same vpc. 
    - what does that mean for you? (or me) - well it means uploading the data from my local mongoDB is going to be a pain and we're going to use the EC2 instance as a sort of intermediary between us and documentDB
    
- 5) Create a security group so that your EC2 instance can actually connect to your documentDB
- 6) Create your DocumentDB cluster
- 7) Connect your EC2 instance to your Document DB cluster
- 8) Install the Mongo Shell on your EC2 instance
- 9) Manage TLS 
- 10) Test Connection with mongo shell 
- 11) Install Jupyter notebooks and pymongo on EC2 instance 
- 12) SCP your local CSV file to your EC2 instance
- 13) insert your data into documentDB w chunksize probably since you'll be choosing a cheap instance

For Steps 1 - 10 you can follow this document [https://docs.aws.amazon.com/documentdb/latest/developerguide/connect-ec2.html](https://docs.aws.amazon.com/documentdb/latest/developerguide/connect-ec2.html)

It will get you up and running with the services 

Step 11

- You can follow parts of this tutorial - https://chrisalbon.com/code/aws/basics/run_project_jupyter_on_amazon_ec2/
- _right click_ and *COPY LINK ADDRESS* https://www.anaconda.com/products/individual
- You won't need to set up a whole new virtual env as the reason for this ec2 instance is just to communicate with your docDB

Step 12

Now that you have your docDB cluster set up, your ec2 instance set up and theyre in the same vpc we can now start sending our data over.

- SCP/SFTP: You will need to ssh into your ec2 instance using the special .epm/.cer key that you downloaded following steps 1-10.

- Download save your mongoDB database as a csv
    - can be done through pd.DataFrame.from_records(db.collection.find()).to_csv('{name}.csv', index=False)

- 'Put' your csv into your ec2 instance. The three datasets I was putting were ~7k rows, ~3million rows, ~3million rows. I also used the free tier instance but if its just for the file transfer and you don't want to do anything with chunksize i'd say just provision a larger instance for the file transfer then shut it down

- Write your data to docDB, [how to programatically connect to docDB](https://docs.aws.amazon.com/documentdb/latest/developerguide/connect_programmatically.html)
    - if you provisioned the smallest instance then in your script to be memory efficient you will need to create an iterator for your dataset that reads and writes chunks at a time.
    

## Why I'm trying DynamoDB?

Using docDB as our primary database for our entire NBA ecosystem will slightly complicate any apps I deploy to EB regarding VPCs/Security Groups etc. Currently I haven't figured out how to connect to docDB from my docker container but as I was figuring this out guess what happened?  

They charged me 9$ (Freedom not Maple) for hosting my DB for a day ... infact it was overnight! once I saw this bill I said screw it i'm moving to dynamoDB for serverless hosting as they will charge depending on traffic! 

## Setting up DynamoDB

Setting up DynamoDB is A LOT easier than before. In fact it can be done through your local machine. 

#### Prerequisites 

- Have the Python SDK for AWS installed on your pc (it's called boto3, I don't know why though)
- Obviously have an AWS account lol 

_note_: the api is a little weird if you haven't seen stuff like this before, and i'm not too entirely comfortable with it either however, working through examples and being able to perform CRUD and some batch operations should get you up and running enough to go out and debug examples on your own

For our example below we will be uploading the same .csv files as in our docDB walk through. Specifically: game_log and historical_pbp_modelled

In [1]:
#define your imports
import pandas as pd
import boto3

In [2]:
#you make your connection to dynamoDB through the boto3 resources method
dynamoDB = boto3.resource('dynamodb', region_name = 'us-east-2')

In [ ]:
#you might have to include your accessID and secret ID from your IAM role if it isnt automatically detected
#in that case you just set up a session 
access_id = 'wouldnt you like to know'
secret_id = 'I aint telling you'

session = boto3.Session(access_id, secret_id)
dynamoDB = session.resource('dynamoDB', region_name = 'us-east-2')

Now that we have the dynamoDB object to interact with we can start to create tables but before we do that you need to know how you are going to use the table. Knowing this will allow us to design the appropriate index structure to for efficent lookups.

An Example below:

In [5]:
game_log = pd.read_csv('../NBA/Data/game_log.csv').drop('_id', axis = 1)
game_log.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Home,Away,home_team_win
0,22015,1610612744,GSW,Golden State Warriors,21500003,2015-10-27,GSW vs. NOP,W,240,41,...,8,7,20,29,111,16,1,GSW,NOP,1.0
1,22015,1610612741,CHI,Chicago Bulls,21500002,2015-10-27,CHI vs. CLE,W,240,37,...,6,10,13,22,97,2,1,CHI,CLE,1.0
2,22015,1610612737,ATL,Atlanta Hawks,21500001,2015-10-27,ATL vs. DET,L,240,37,...,9,4,15,25,94,-12,1,ATL,DET,0.0
3,22015,1610612747,LAL,Los Angeles Lakers,21500017,2015-10-28,LAL vs. MIN,L,240,35,...,2,4,14,29,111,-1,1,LAL,MIN,0.0
4,22015,1610612756,PHX,Phoenix Suns,21500014,2015-10-28,PHX vs. DAL,L,240,34,...,3,3,18,30,95,-16,1,PHX,DAL,0.0


For our in game win probability use case we want the user:
- 1) To be able to select a date
- 2) Pick the games from this date they want to see the win probability for.

To achieve this with a regular SQL db, amazon RDS, mongo/noSQL db, docDB is pretty simple but with dynamoDB it can get a little tricky. The tricky part is in the inital learning but after that it becomes a lot clearer.


## DynamoDB key structure

- Hash Key(aka Partition Key): 
    - This is a required key
    - _Single_ Tables are defined as tables that ONLY have a Hash Key. 
        - A single table is something you generally want to avoid with dynamoDB because it limits you to one read at a time as it **only** enables the get_item method. I'm gyuessing this isn't what you want and that you'll probably want to query your data
    - This is the main key and _must_ be unique


unless you have a ...

- Sort Key(aka Range key): 
    - This is an optional second key but can be used in conjunction with the Hash key to create a _Composite_ index where your Hash/Sort Key pair HAS to be unique. 
    - Having a composite key will allow us to perform more complex methods to retrieve data from our dynamoDB including Query and Scan operations.
    
For our use case we will create a composite key consisting of our GAME_DATE as our HASH key and GAME_ID as our SORT key. 

The Reason I have mentioned these BEFORE we create the table is because once created we can't change it. Annoying, but is it really a bug or feature lol.


[Further Reading on this topic](https://dynobase.dev/dynamodb-keys/#:~:text=Is%20it%20possible%20to%20change,then%20remove%20the%20first%20table.) 

In [ ]:
# Creating the table

game_log_ddb = dynamodb.create_table(
        TableName='game_log',
        KeySchema=[
            {
                'AttributeName': 'GAME_DATE',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'GAME_ID',
                'KeyType': 'RANGE' # Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'GAME_DATE',
                'AttributeType': 'S' #string
            },
            {
                'AttributeName': 'GAME_ID',
                'AttributeType': 'N' #number
            },

        ],
    BillingMode= 'PAY_PER_REQUEST', 
    ) 

# A quick note on Billing Mode: PAY_PER_REQUEST provisions resources depending on traffic
# and is useful if you don't know the frequency at which your db will get called
# otherwise you can pre provision resources which will throttle read and write speeds
# but for me since this app is a personal project I am going with PAY_PER_REQUEST for fast write speeds 
# during intial upload and I'll only be querying this db a handful of times a week. 

# Time to write our local data. 

We cant just dump our csv in like mongoDB with the insert_many method instead we will have to create a _batch_writer_ object and load each row indvidually but if you selected BillingMode=PAY_PER_REQUEST then the data even if its in a for loop will be written in parallely. 

Additionally, floats arent compatible so you'll have to convert all your floats to Decimal (which I didn't even know was a type until getting to this step)

In [ ]:
game_log_dict = game_log[~game_log['home_team_win'].isna()].to_dict(orient = 'records')
#how we convert float to decimal
game_log_json = [json.loads(json.dumps(item), parse_float=Decimal) for item in game_log_dict]

In [ ]:
from decimal import Decimal
import json

#if a float column has a nan you will have problems - again this is annoying, alternatively you can convert your
#data entirely to strings and then write it but Im going to get rid of the few rows that have nan in any float columns
#I am working with


with game_log_ddb.batch_writer() as batch:
    for i in range(len(game_log_json)):
        batch.put_item(Item = game_log_json[i])
        
        
#should take under a minute for the ~7500 rows I have in this dataset.

Doing this for the data that has the win probabilities ~3million rows took between 2-3 hours.

## How do you query the data? 

In [9]:
from boto3.dynamodb.conditions import Key

#Example query. GAME_DATE is our HASH KEY
data = game_log_ddb.query(
    KeyConditionExpression=Key('GAME_DATE').eq('2022-01-01')
)

In [ ]:
#if you want to make a query with both GAME_DATE and GAME_ID then  

data = game_log_ddb.query(
    KeyConditionExpression=Key('GAME_DATE').eq(game_date) & Key('GAME_ID').eq(game_id)
)

## Global Secondary Indexes

Okay, so you've moved your data from local to dynamoDB, you have everything wokring and have connected dynamoDB to your apis but as you move on with your project you realise that you need to use another column as a key to execute queries. What do you do? We can't reset the Partition key and Sort key that we originally set up + you need to use them anyways for your first use case. 

if only we could create another partition key and sort key? 
You can! and you can create/delete them even after the database has been created. They are called GSI's or Global Secondary Indexes and they allow you to perform queries like you would with your original partition and sort key!



In [ ]:
resp = dynamoDB_client.update_table(
    TableName="game_log",
    # Any attributes used in your new global secondary index must be declared in AttributeDefinitions
    AttributeDefinitions=[
        {
            "AttributeName": "GAME_ID",
            "AttributeType": "N"
        },
        {
            "AttributeName": "GAME_DATE",
            "AttributeType": "S"
        }
    ],
    # This is where you add, update, or delete any global secondary indexes on your table.
    GlobalSecondaryIndexUpdates=[
        {
            "Create": {
                # You need to name your index and specifically refer to it when using it for queries.
                "IndexName": "GameIdIndex",
                # Like the table itself, you need to specify the key schema for an index.
                # For a global secondary index, you can use a simple or composite key schema.
                "KeySchema": [
                    {
                        "AttributeName": "GAME_ID",
                        "KeyType": "HASH"
                    },
                    {
                        "AttributeName": "GAME_DATE",
                        "KeyType": "RANGE"
                    }
                ],
                # You can choose to copy only specific attributes from the original item into the index.
                # You might want to copy only a few attributes to save space.
                "Projection": {
                    "ProjectionType": "ALL"
                }
            }
        }
    ],
)

from boto3.dynamodb.conditions import Key

game_log_db.query(IndexName="GameIdIndex",
                  KeyConditionExpression=Key('GAME_ID').eq(22100540))

If you see here I created a global secondary index that reversed my original key structure just incase I wanted to search for games based purely of game_id!

###### These articles are subject to revision because learning is a journey 